#Introdiction

The objective of this competion is to use historical loadn application data to predict whether or not an applicatn will be ale o replay a loan. This is a standard supervised classification task :

- Supervised : The labels ar eincluded in the training data and teh goal is to train a  model to learn to predict the labels fro the features.
- Classification : The label is a binary viriable, 0(will repay loan on time) or 1(will have difficulty repaying loan)



#Data

- application_train/application_test : identified by the feature ***SK_Id_CURR***.
 The training application data comes with the target indicating 0  the loan was repaid or 1 : the loan was not repaid.

 - bureau : client's previous credits from other financial institutions.

 - bureau_balance : monthly data about the previous credits in bureau.

 - previous_application : previous applications for loans at home credit of clients who have loans in the application data.

 - POS_CASH_BALACE : monthly data about previous point of sale or cash loans

 - credit_card_balance : monthly data about previous credit cards clients have had with home credit.

 - installments_payment : payment history for previous loans at home credit.

 

#Metric : ROC AUC

A single line on the graph that the curve for a single mode, and movement along a line indicates chaningn the threshild used for calssifying a positive instance, The threshold starts a t 0 in the upper right to and goes to 1 in the lower left.


The Area Under the Curve(AUC) explains itself by its name. It is simply the areunder the ROC curve. This metric is between 0 and 1 with a better model scoring higher. Model that simply guesses at random will have an ROC AUC of 0.5.



Follow-up notebooks

- Manual Feature Engineering 1
- Manual Feature Engineering 2
- Introduction to Automated FEature Engineering
- Advanced Automated Feature Engineering
- Feature Sleection
- Intro to Model Tuning : Grid and Random Search
- Automated Model Tuning
- Model Tuning Results

#Imports

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

import os

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

##Read in Data

first, we can list all the available data fiels. There are a total of 9 files : 1 main file for training (with target) 1 main file for testing(without the target), 1 example submission file, and 6 other files containing additional information about each loan.

In [ ]:
print(os.listdir('../input/'))

In [ ]:
#Training data

app_train = pd.read_Csv('../input/application_train.csv')
print('Training data shape :', app_train.shape)

app_train.head()

In [ ]:
#Testing data features

app_test= pd.read_csv('../input/application_test.csv')
print('Testing data shape :', app_test.shape)
app_test.head()

#EDA

EDA is an open ended process where we calculate statistics and make figures to find trends, anomalies, patterns, or relationships within the data. The goal of EDA is to learn what our data can tell us. It genereally starts out with a high level overview, then narrows in to specific areas as we find intriguing areas of the data. The findings may be interesting in their own righrm or they can used to inform our modeling choices, such as by helping us decide which features to use.

Examine the Distribution of the Target Column

The target is what we are asked to predict : either a 0 for the loan was repaid on time, or  a 1indicating the client had payment difficulties. We can first examine the number of loans falling into each category.

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
app_train['TARGET'].astype(int).plot.hist()

##Examine Missing values

In [ ]:
def missing_values_table(df) :
  mis_val = df.isnull().sum()

  mis_val_percent = 100*df.isnull().sum() / len(df)

  mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

  #rename the col
  mis_val_table_ren_col = mis_val_table.rename(columns={0 : 'Missing values', 1 : '% of total values'})

  #Sort the table by percentage of missing descending
  mis_val_table_ren_columns = mis_val_table_ren_col[mis_val_table_ren_col.iloc[:,1] !=0].sort_values('% of total values', ascending=False).round(1)

  #Print some summary information
  print('Your selected dataframe has ' + str(df.shape[1]) + 'columns.\n' + 'There are ' + str(mis_val_table_ren_columns.shape[0]) + 'columns that have missing values')

  #Return the dataframe with missing information
  return mis_val_table_ren_columns

In [ ]:
#Missing values statistics

missing_values = missing_values_table(app_train)
missing_values.head(20)

Qhwn it comes time to build our machine learning models, we will have to fill in these missing vlaues. In later work, we will use models such as XGBoost that can handle missing values with no need for imputation.

Another option would be to drop columns with a high percentage of missing values, although it is impossible to know ahead of time i these columns will be hilpful to our model. Therefore, we will keep all of the columns for now.

##Column Types

Let's look at the #cols of each data type. int and float are numeric variables. objs cols contain str and are categorical features.

In [ ]:
app_train.dtypes.value_counts()

In [ ]:
app_train.select_dtypes('object').apply(pd.Series.nunique,axis=0 )

#nunique : Return number of unique elements in the object.

Most of the categorical variables have a relatibely small number of unique entries.

We will need to find a way to deal with these categorical variables.

##Encoding Categorical Variables

Before we go any further, we need to deal with pesky categorical variables. A machine learning model unfortunately cannot deal with categorical variables(such as LGBM). Therefore, we have to find a way to encode these variables as numbers bofore hadling them off to the model. There are two main wayy to carry out this process

1. Label encoding : assign each unique category ina categorical variable with an integer., No new columns are created. 

2. One-hot encoding : create a new col for each unique category in a categorical variable. Each observation recieves a 1 in the column for its corresponding category and a 0 i all other new columns.

The problem with label encoding is that it gives the categories an arbitrary ordering. The value assigned to each of the categories in rando and does not reflect any ingerent aspect of the category. it will may cause a confusion btw programmer and data scientists.  Therefore, when we perform label encoding, the model might use the relative value of the feature ti assign weights which is not what we want. // for more than 2 unique categories, one-hot-encoding is the safe option.

The onyl downside to one-hot-encoding is that #features can explode with categorical variables with many categories. To deal with this, we can perform one-hot encoding followed by PCa or other dimensionality reduction methods

###Label Endocing and one-hot encoding

LEt's implemet the policy described above : for any categorical variables with 2 unique categories, we will use label encoding, and for any categorical variable with more than 2 unique categories, we will use one-hot encoding.

In [ ]:
#create a label encoder object

le = LabelEncoder()

le_count=0

#Iterate through the columns

for col in app_train :
  if app_train[col].dtype == 'object' :
    #if 2 or fewer unique categories
    if len(list(app_train[col].unique())) <= 2 :
      le.fit(app_train[col])

      #Transform both training and testing data
      app_train[col] = le.transform(app_train[col])
      app_test[col] = le.transform(app_test[col])

      #Keep track of how many columns were label encoded
      le_count+=1

print(' {} columns were label encoded.'.format(le_count))

In [ ]:
#one-hot encoding of categorical variables

app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

print('Training FEature shape : ', app_train.shape)
print('Testing Feature shape : ', app_test.shape)



###Aligning Training and Testing Data

There need to be the same features (cols) in both the training and testing data.

To ***remove the columns in the training data that are not in the testing data***, we need to align the dataframes.

First, we extract the target column from the training data(bcuz this si not in the testing data but we need to keep this information). When we do the align we must make sure to set axis=1 to align the df based on the columns and not on the rows

In [ ]:
train_labels = app_Train['TARGET']
#Align the training and testing data, keep only columns present in both df

app_train, app_test = app_train.align(app_test, join = 'inner', axis=1)

#Add the target back in
app_train['TARGET'] = train_labels

print('Training feature shape :', app_train.shape)
print('Testing feature shape :' , app_test.shape)


###Back to Exploratory Data Analysis

####Anomalies

In [ ]:
(app_train['DAYS_BIRTH'] / -365).describe()

In [ ]:
app_train['DAYS_EMPLOYED'].describe()

In [ ]:
app_train['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram')

plt.xlabel('Days Employment')

In [ ]:
anom = app_train[app_train['DAYS_EMPLOYED'] == 365243]
non_anom = app_train[app_train['DAYS_EMPLOYED'] != 365243]
print('The Non-anomalies default on %.2f%% of loans'% (100*non_anom['TARGET'].mean()))

print('The anomalies default on %.2f%% of loans' % (100*anom['TARGET'].mean()))

print('There are %d anomalous days of employment' % (len(anom)))

One of the safest approaches is just to set the anomalies to a missing value and then have them filled in (using imputation) vefore machine learning. In this case, since all the anomalies have the exact same value, we want to fill them in with the same value in case all of these loans share something in common. 

In this solution, we will fill in the anomalous values with np.NaN and then create a new boolean cols indicating whether or not the value was anomalous.

In [ ]:
#Create an anomalous flag column

app_train['DAYS_EMPLOYED_ANOM'] = app_train['DAYS_EMPLOYED'] == 365243

#Replace the anomalous values with nan

app_train['DAYS_EMPLOYED'].replace({365243 : np.NaN}, inplace=True)

app_train['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram')
plt.xlabel('Days Employment')

In [ ]:
#Create an anomalous flag column

app_test['DAYS_EMPLOYED_ANOM'] = app_test['DAYS_EMPLOYED'] == 365243

#Replace the anomalous values with nan

app_test['DAYS_EMPLOYED'].replace({365243 : np.NaN}, inplace=True)

app_test['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram')
plt.xlabel('Days Employment')

###Correlations



In [ ]:
# find correlations with the target and sort

correlations = app_train.corr()['TARGET'].sort_values()

#Display corrrelations

print('Most Posirive Correlation : \n', correlations.tail())
print('\n Most Negative Correlation : ', correlations.head())

# sns.heatmap(correlations, annot=True, fmt='2.f', square=True blah blabh blah~~)

Effect of Age on Repayment

In [ ]:
#find the correlation of the positive days since birth and target

app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])

app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

In [ ]:
#Set the style of plots

plt.style.use('fivethirtyeight')

#plot the distribution of ages in years
plt.hist((app_train['DAYS_BIRTH' / 365), edgecolor='k', bins=25])
plt.title('Age of Client')
plt.xlabel('Age(years')
plt.ylabel('Count')


In [ ]:
plt.figure(figsize=(10,8))

sns.kdeplot(app_train.loc[app_train['TARGET']==0, 'DAYS_BIRTH'] /365, label = 'target==0')
sns.kdeplot(app_train.loc[app_train['TARGET'] ==1, 'DAYS_BIRTH'] / 365, label = 'target==0')

plt.xlabel('Age')
plt.ylabel('Distribution')
plt.title('Distribusion of Ages')

In [ ]:
# AGe information into a separate dataframe

age_data = app.train[ ['TARGET','DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / 365

#bin the age data
age_data['YESRS_BINNED'] = pd.qcut(age_data['YEARS_BRITH'], bins=np.linspace(2070,num=11))

age_data.head()

In [ ]:
#Group by the bin and calculate averages

age_groups = age_data.groupby('YEARS_BINNED').mean()

age_groups

In [ ]:
plt.figure(figsize=(8,8))

plt.bar(age_groups.index.astype(str), 100*age_groups['TARGET'])

plt.xticks(rotation=75)
plt.xlabel('Age Group(yrs')
plt.ylabel('Failure to Repay')
plt.title('Failure to Repay by age group')

Exterior Sources

3 variables with the strongest neg corr with the target are 'ext_souce_1','ext_source_2','ext_source_3'

In [ ]:
# Extract the EXT_SOURCE variables and show correlations

ext_data = app_train[ [ 'TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]

ext_data_corrs= ext_data.corr()
ext_data_corrs

In [ ]:
plt.figure(figsize=(8,6))

sns.heatmap(ext_data_corrs, cmap = plt.cm.RdYlBu_r, vmin=-0.2, annot=True, vamx=0.6)

plt.title('Correlation Heatmap')

In [ ]:
plt.figure(figsize=(10,12))

#iterate throug the source

for i , source in enumerate(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']) :
  plt.subplot(3,1,i)

  sns.kdeplot(app_train.loc[app_train['TARGET'] ==0, source], label='target==0')
  sns.kdeplot(app_train.loc[app_train['TARGET'] ==1, source], label='target==1')

  plt.xlabel('%s' % source)
  plt.ylabel('Density')
  plt.title('Distribution of %s by Target values' % source)

plt.tight_layout(h_pad=2.5)

Pairs Plot

In [ ]:
# copy the data for plotting

plot_data = ext_data.drop(['DAYS_BRITH']).copy()

#add in the age of the client in years
plot_data['YEARS_BIRTH'] = age_Data['YEARS_BIRTH']

#drop na values and limit to first 100k rows

plot_data = plot_data.dropna().loc[:100000, :]

#function to calculate correlation coefficient between two clumns
def corr_func(x,y, **kwargs) :
  r=np.corrcoef(x,y)[0][1]
  ax = plt.gca()
  ax.annotate('r = {:.2f'.format(r), xy=(.2,.8), xycoords=ax.transAxes, size=20)

#Create the parigrid object
grid= sns.Pairplot(plot_data, sie=3, diag_sharey=False, hue='TARGET', 
                   vars= [x for x in list(plot_data.columns) if x != 'TARGET']) 

#UPPER IS A SCATTER PLOT
grid.map_upper(plt.scatter, alpha=0.2)

#Diagonal is a histogram
grid.map_diag(sns.kdeplot)

#Bottom is density plot
grid.map_lower(sns.kdeplot, cmap=plt.cm.OrRd_r)

plt.subtitle('Ext source and age features pairs plot', size=32, y=1.05)

In this plots, the red indicates loans that were not repaid and the blue are loans that are paid. we can see the different relationships within the data, There dose appear to be a moderate positive linear relationship between the EXT_SOUCE_1 and the DAYS_BIRTH, indicating that this feature may take into account the age of the client.

#Feature Engineering

most parts as the winning models, at least for structured data, all tento to be variants on gradient boosting


- applied machine learning is basically feature engineering




Polynomial Features

in polynomial features, we can create variables such as ext_source_1^2 and ext_source_2^2.

or ext_source_1 x ext_source_2 /

ext_source_1 x ext_source_2^2


The combination of multiple individual variables are called ***interaction terms*** bcuz they capture the interactions between variables. In other words, while two variables by hemselves may not have a strong influence on the target, ***combining them tohether into a single interaction vatiable might show a relationship with the target***.

In [ ]:
poly_features = app_train [['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH','TARGET']]

poly_features_test = app_test [['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]

#imputer for handling missing values

#from sklearn.preprocessing import Imputer
from sklearn.imput import SimpleImputer

imputer = SimpleImputer(strategy='median')

poly_target = poly_features['TARGET']

poly_features = poly_features.drop(['TARGET'])

#Need to impute missing values

poly_features = SimpleImputer.fit_transform(poly_features)
poly_features_test = SimpleImputer.fit_transform(poly_features_test)

from sklearn.preprocessing import PolynomialFeatures

#Create the polynomial object with specified degree

poly_transformer = PolynomialFeatures(degree=3)

In [ ]:
#Train the polynomial features

poly_transformer.fit(poly_features)

#Transform the features
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)

print('Polynomial Features shape : ', poly_features.shape)

This creates a considerable #new features. To get the names we have to use the polynomial features get_feature_names method.

In [ ]:
poly_transformer.get_feature_names(input_features = ['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH'])[:15]

#15개의 최대조합 생성

There are 35 features with individual features raised to powers up to degree 3 and interaction terms. Now, we can see whether any of these new features are correalted with the target.

In [ ]:
#Create a dataframe of the features

poly_features = pd.DataFrame(poly_features, columns = poly_transformer.get_feature_names(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']))

#Add in the target
poly_features['TARGET'] = poly_target

#Find the correlations with the target
poly_corrs = poly_features.corr()['TARGET'].sort_values()

#Display most negtive andn most positive
print(poly_corrs.head())
ptiny(poly_corrs.tail())

Several of the new variables have a greater correlation with the target than the original features. When we build ML models, we can try with and without these features to determine if they actually help the model learn.

We will add these features to a copy of the training and testing data and then evaluate models with and without the features. Many time in ML, the only way to know if an approach will work is to try it out

In [ ]:
#put test featues into df

poly_features_test = pd.DataFrame(poly_features_test, columns=poly_transformer.get_feature_names(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3', 'DAYS_BIRTH']))

#Merge polynomial features into training df
poly_features['SK_ID_CURR'] = app_train['SK_ID_CURR']
app_train_poly = app_train.merge(poly_features, on ='SK_ID_CURR', how='left')

#Merge polynomial features into testing df
poly_features['SK_ID_CURR'] = app_test['SK_ID_CURR']
app_test_poly = app_test.merge(poly_features_test, on='SK_ID_CURR', how='left')

#Align the df
app_train_poly, app_test_poly = app_train.align(app_test_poly, join='inner', axis=1)

#Print out the new shapes
print('Training data with polynomial features shape :', app_train_poly.shape)
print('test data with polynomial features shape :', app_test_poly.shape)

##Domain Knowledge Features

we could call this 'attempts at applying limited financial knowledge'. In this frame of mind, we cna make a couple features that attempt to capture what we think may be important for telling whether a client will default on a loan.

- CREDIT_INCOM_PERCENT : % of the credit amount relative to a client's income
- ANNUITY_INCOME_PERCENT : % of the loadn annuity
- CREDIT_TERM : len of the payment in month
- DAYS_EMPLOYED_PERCENT : % of the days employed relative to the client's age

In [ ]:
app_train_domain = app_train.copy()
app_test_domain=app_test.copy()

app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['ANNUITY_INCOME_PERCEN'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['CREDIT_TERM'] = app_train.domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_train_domain['DAYS_EMPLOYED_PERCENT']=app_train_domain['DAYS_EMPLYED'] / app_train_domain['DAYS_BIRTH']

In [ ]:
app_test_domain['CREDIT_INCOME_PERCENT'] = app_test_domain['AMT_CREDIT'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['ANNUITY_INCOME_PERCEN'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['CREDIT_TERM'] = app_test.domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']
app_test_domain['DAYS_EMPLOYED_PERCENT']=app_test_domain['DAYS_EMPLYED'] / app_test_domain['DAYS_BIRTH']

Visualize new variables

In [ ]:
plt.figure(figsize(12,20))

for i, feature in enumerate(['CREDIT_INCOME_PERCENT','ANNUITY_INCOME_PERCENT','CREDIT_TERM','DAYS_EMPLOYED_PERCENT']) :
  plt.subplot(4,1,i+1)
  #plot that repaid loans

  sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] ==0, feature], label = 'target==0')
  sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] ==1, feature], label = 'target==1')

  #Label the plots
  plt.title('Distribution of %s by Target values' % feature)
  plt.xlabel('%s' % feature)
  ply.ylabel('Density')

plt.tight_layout(h_pad=2.5)

#Baseline

For a naive baseline, we could guess the same value for all examples on the testing set. We are asked to predict the probability of not repaying the loan, so if we are entirely unsure, we would guess .5 for all observations on the test set.

This will get us a Reciever Operating Characteristic Area under the Cuve (AUCROC) of .5 in the comptition

Since we already know what score we are going to get, we don't really need to make a naive baseline guess.

##Logistic Regression Implementation

To get a baseline, we will use all of the features after encoding the categorical vaiables. We will preprocess the data by filling in the missing values and normalizing the range of the features (feature scaling). The following code performs both of these preprocessing steps.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, Imputer

#drop the target from the training data
if 'TARGET' in app_train :
  trian=app_train.drop(['TARGET'])

else :
  train=app_train.copy()


#Feature names
features = list(train.columns)

test = app_test.copy()

#MEdian imputation of missing values
imputer = SimpleImputer(strategy='median')

#Scale each feature to 0-1
scaler=MinMaxScaler(feature_range=(0,1))

#fit on the training data
imputer.fit(train)

#Transform both training and testing data
train=imputer.transform(train)
test=imputer.transform(test)

#Repeat with the scaler
scaler.fit(train)
train=scaler.transform(train)
test=scaler.transform(test)

print('Training data shape : ', train.shape)
print('Test data shape : ', test.shape)


We will use LogisticRegression from sklearn for our first model. The only change we will make from the default model settings is to lower the regularization parameter, C, which controls the amount of overitting (a lower value should decrease overfitting). This will get us slightly better results than the default LogisticRegression, but it still will set a low bar for any future models.

Here we use the familiar Scikit-Learn modeling syntax : we first create the model, then we train the model using.

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(C=0.0001)

log_reg.fit(train, train_labels)

In [ ]:
#make predictions
#make sure to select the second column only
log_reg_pred = log_reg.predict_proba(test)[:,1]

The predictions must be in the format shown in the sample_submission.csv file, where there are only two columns : SK_ID_CURR and TARGET We will create a df in this format from the test set and the predictions called submit.

In [ ]:
#Submission df
submit = app_test[ ['SK_ID_CURR']]
submit['TARGET']=log_reg_pred

submit.head()

The predictions represent a probability between 0 and 1 that the loan will not be repaid. If we were using these predictions to classify applicatns, we couls set a probablity threshold for determining that a loan is risky.

In [ ]:
#Save the submission to a csv file

submit.to_csv('log_reg_baseline.csv', index=False)

##Improved Model : Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#make the random forest classifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42, verbose=1, n_jobs=-1)

In [ ]:
#Train on the training data

random_forest.fit(train, train_labels)

#Extract feature importances
feature_importance_values=random_forest.feature_importances_
feature_importances = pd.DataFrame({'feature' : features, 'importrnacec':feature_importance_values})

#make predictions on the test data
predictions=random_forest.predict_proba(test)[:,1]


In [ ]:
#make a submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET']=predictions

submit.to_csv('random_forest_baseline.csv', index=False)

##Make Predictions using Engineered Features

In [ ]:
poly_features_names =list(app_train_poly.columns)

#Impute the polynomial features
imputer=SimpleImputer(strategy='median')

poly_features = imputer.fit_transform(app_train_poly)
poly_features_test = imputer.transform(app_test_poly)

#Scale the polynomial features
scaler = MinMaxScaler(feature_range=(0,1))

poly_features = scaler.fit_transform(poly_features)
poly_features_test = scaler.transform(poly_features_test)

random_forest_poly = RandomForestClassifier(n_estimators=100,random_state=42, verbose=1, n_jobs=-1)



In [ ]:
#Train on the training data
random_forest_poly.fit(poly_features, train_labels)

#make predictions on the test data
predictions = random_forest_poly.predict_proba(poly_features_test)[:,1]

In [ ]:
submit=app_test[ ['SK_ID_CURR']]
submit['TARGET'] = predictions

submit.to_csv('random_forest_baseline_engineered.csv', index=False)

###Testing Domain Features

In [ ]:
app_train_domain = app_train_domain.drop('TARGET')

domain_features_names = list(app_train_domain.columns)

imputer= SimpleImputer(strategy='median')

domain_features = imputer.fit_transform(app_train_domain)
domain_features_test=imputer.transform(app_test_domain)

#Scale the domainnomial features
scaler = MinMaxScaler(feature_range=(0,1))

domain_features = scaler.fit_transform(domain_features)
domain_features_test = scaler.transform(domain_features_test)

random_forest_domain = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)

#train on the training data
random_forest_domain.fit(domain_features, train_labels)

#Extract feature importances
feature_importance_values_domain = random_forest_domain.feature_importances_
feature_importrances_domain = pd.DataFrame({'feature':domain_features_names,'importance' : feature_importances_values_domain})

#make predictions on the test data
predictions = random_forest_domain.predict_proba(domain_features_test)[:,1]

In [ ]:
# Make a submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = predictions

# Save the submission dataframe
submit.to_csv('random_forest_baseline_domain.csv', index = False)

Model Interpretation : Feature importances

In [ ]:
def plot_feature_importances(df) :
  #sort features according to importance
  df=df.sort_values('importance', ascending=False).reset_index()

  #Normalize the feature importances to add up to one
  df['importance_normalized'] = df['importance']/df['importnace'].sum()

  #make a horizontal bar chart of feature importances
  plt.figure(figsize=(10,6))
  ax=plt.subplot()

  ax.barh(list(reversed(list(df.index[:15]))),
               df['importnace_normalized'].head(15),
               align = 'center' edgecolor = 'k' )
  
  #Set the yticks and labels
  ax.set_yticks(list(reversed(list(df.index[:15]))))
  ax.set_yticklabels(df['feature'].head(15))

  #plot labeling
  plt.xlabel('Normalized Importance')
  plt.title('Feature importrances')
  plt.show()

  return df

In [ ]:
feature_importances_sorted = plot_feature_importances(feature_importnaces)

In [ ]:
feature_importances_domain_sorted = plot_feature_importances(feature_importances_domain)

#Conclusion


In short, we followed the general outline of a ML Project :

1. Understand the problem and the data
2. Data cleaning and formatting (this was mostly done for us)
3. EDA
4. Baseline Model
5. Improved model
6. Model Interpretation (just a little)

#LGBM

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metric import roc_auc_score
import lightgbm as lgb
import gc

def model(features, test_features, encoding = 'ohe', n_folds=5) : 
  '''train and test a light gradient boosting model using cross validation.

  Parameters 
  1. encoding (str, default = 'ohe')
  : method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label
  ------
  return
  1. submission(pd.DataFrame)
  : df with 'SK_ID_CURR' ad 'TARGET proba that predicted by the model.
  2. feature_improtances
  : df with the feature importances from the model
  3. valid_metrics
  : df with training and validation metrics for each fold and overall
  '''

  #Extract the ids
  train_ids = features['SK_ID_CURR']
  test_ids = test_features['SK_ID_CURR']

  #Extract the labels for training
  labels = features['TARGET']

  #Remove the ids and target
  features = features.drop(['SK_ID_CURR', 'TARGET'])
  test_features = test_features.drop(['SK_ID_CURR'])

  #ohe
  if encoding = 'ohe':
    features = pd.get_dummies(features)
    test_features = pd.get_dummies(test_features)

    #align the df by the columns
    features, test_features = features.alien(test_features, join='inner', axis=1)

    #No categorical indices to records
    cat_indices = 'auto'
  #integer label encoding
  elif encoling ==' le':
    label_encoder = LabelEncoder()

    #List for storing categorical indices
    cat_indices =[]

    #Iterate through each column
    for i, col in enumerate(features) :
      if features[col].dtype == 'object' :
        #Map the categorical features to integers
        features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape(-1,) )

        #record the categorical indices
        cat_indices.append(i)

  #Catch error if label encoding scheme is not valie
  else :
    raise ValueError('Encoding must be either 'ohe' or 'le'')
  
  print('Training data shape : ', features.shape )
  print('Test data shape : ', test_features.shape )


  feature_names = list(features.columns)

  #Convert to np arrays

  features = np.array(features)
  test_features = np.array(test_features)

  #Create the kfold object

  k_fold=KFold(n_splits =n_folds, shuffle=False, random_state=42)

  #empty array for feature importances
  feature_importance_values = np.zeroes(len(feature_names))

  #empty array for test predictions
  test_predictions = np.zeroes(test_features.shape[0])

  #empty array for out of fold validation predictions
  out_of_fold = np.zeroes(features.shape[0])

  #list for recording validation and training scores
  valid_scores=[]
  train_scores=[]

  #Iterate through each fold
  for train_indices, valid_indices in k_fold.split(features) :

    #Training data for the fold
    train_features, train_labels = features[train_indices], labels[train_indices]
    #Validation data for the fold
    valid_features, valid_labels = features[valid_indcies], labels[valid_indices]

    #Create the model
    model = lgb.LGBMClassifier(n_estimators=1000, objective='binary', class_weight='balanced', learning_rate=0.1,
                               reg_alpha=0.1, reg_lambda=0.1, subsample=0.8, n_jobs=-1, random_state=42)
    
    #Train the model
    model.fit(train_features, train_labels, eval_metric='auc',
              eval_set=[(valid_features, valid_labels), (train_features, train_labels)],
              eval_names = ['valid','train'],
              categorical_feature = cat_indices,
              early_stopping_rounds=100, verbose=200)
    best_iteration = model.best_iteration_

    #Record the feature importances
    feature_importrnace_values += model.feature_importances_ / k_fold.n_splits

    #Make predictions
    test_predictions += model.predict_proba(test_features, num_iternation = best_iterration)[:,1]/k_fold.n_splits

    #Record the best score
    valid_score=model.best_score_['valid']['auc']
    train_score = model.best_score_['train']['auc']

    valid_score.append(valid_score)
    train_score.appned(train_score)

    #Clean up memory
    gc.enable()
    del model, train_feature, valid_features
    gc.collect()

    #make the submission df
  submission = pd.DataFrame({'SK_ID_CURR':test_ids, 'TARGET':test_predictions})

    #make the feature importance dataframe
  feature_importances = pd.DataFrame({'feature' : feature_names, 'importance' : feature_importance_values})

    #Overall validation score
  valid_auc = roc_auc_score(labels, out_of_fold)


    #Add the overall scores to the metrics
  valid_scores.append(valid_auc)
  train_scores.append(np.mean(train_scores))

    #Needed for creating dataframe of validation scores
  fold_names = list(range(n_folds))
  folds_names.append('overall')

    #Dataframe of validation scores
  metrics = pd.DataFrame({'fold' : fold_names,
                            'train' :  train_scores,
                            'valid' : valid_scores})

  return submission, feature_importrnaces, metrics
            

In [ ]:
submission, fi, metrics = model(app_train, app_test)
print('baseline metrics')
print(metrics)

In [ ]:
fi_sorted = plot_feature_importances(fi)

In [ ]:
submission.to_csv('baseline_lgb.csv', index=False)

In [ ]:
app_train_domain['TARGET'] = train_labels

#Test the domain knowledge features
submission_domain, fi_domain, metrics_domain = model(app_train_domain, app_test_domain)

print('Baseline with domain knowledge features metrics')
print('metrics_domain')

In [ ]:
fi_sorted = plot_feature_importances(fi_domain)

In [ ]:
submission_domain.to_csv('baseline_lgb_domain_features.csv', index=False)